# Travel Agent Planner with AgentNeo Integration

This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with a Travel Agent Planner system. AgentNeo provides seamless tracing capabilities for both function calls and AI model interactions, allowing for comprehensive analysis and debugging of complex systems.


# Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [32]:
import os
import requests
from dotenv import load_dotenv
from litellm import completion
import openai
from openai import OpenAI

In [33]:
# Load environment variables
load_dotenv("/path/to/your/.env")


# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")


openai key fetch sucessfully: True


In [34]:
# Initialize AgentNeo Package


from agentneo import AgentNeo, Tracer, launch_dashboard
# Initialize AgentNeo session
# Create project
neo_session = AgentNeo(session_name="ai_travel_agent_session18")

project_name = "ai_travel_agent_demo18"

try:
    neo_session.create_project(project_name=project_name)
    print("Project created successfully")
except:
    neo_session.connect_project(project_name=project_name)
    print("Project connected successfully")
# Start tracing

tracer = Tracer(session=neo_session, log_file_path="ai_travel_agent_18.json")
tracer.start()

Project connected successfully
Project 'ai_travel_agent_demo18' found.
Tracing Started.


# Travel Agent Tools
Now, let's define our Travel Agent Planner Tools with AgentNeo integration.

In [35]:
@tracer.trace_llm(name="llm_call")
def llm_call(prompt, max_tokens=512, model="gpt-4o-mini"):
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.7,
    )

    return response.choices[0].message.content.strip()


# Tools outside agents
@tracer.trace_tool(name="weather_tool")
def weather_tool(destination):

    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    params = {"q": destination, "appid": api_key, "units": "metric"}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]

        return f"{weather_description.capitalize()}, {temperature:.1f}°C"
    except requests.RequestException:
        return "Weather data not available."


@tracer.trace_tool(name="currency_converter_tool")
def currency_converter_tool(amount, from_currency, to_currency):
    api_key = os.environ.get("EXCHANGERATE_API_KEY")
    base_url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"

    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()

        if data["result"] == "success":
            rate = data["conversion_rate"]
            return amount * rate
        else:
            return None
    except requests.RequestException:
        return None


@tracer.trace_tool(name="flight_price_estimator_tool")
def flight_price_estimator_tool(origin, destination):
    # This is a mock function. In a real scenario, you'd integrate with a flight API.
    api_key = os.environ.get("FLIGHT_API_KEY")
    # Implement actual API call here
    return f"Estimated price from {origin} to {destination}: $500-$1000"


# Agent with persona
@tracer.trace_agent(name="itinerary_agent")
class ItineraryAgent:
    def __init__(self, persona="Itinerary Agent"):
        self.persona = persona

    def plan_itinerary(self, user_preferences, duration=3):
        itinerary_prompt = f"""
You are a travel expert named {self.persona}.
Based on the following user preferences, create a {duration}-day travel itinerary.

User Preferences:
{user_preferences}

Itinerary:
"""
        return llm_call(itinerary_prompt, max_tokens=512)





# Running the Analysis
Now let's create a  main function for our Travel Agent Planner which recall all tools 

In [36]:
# Main function
@tracer.trace_agent(name="travel_agent")
def travel_agent():
    print("Welcome to the Personalized Travel Planner!\n")

    # Get user input
    user_input = input("Please describe your ideal vacation: ")

    # Extract preferences
    preferences_prompt = f"""
Extract key travel preferences from the following user input:
"{user_input}"

Please provide the extracted information in this format:
Destination:
Activities:
Budget:
Duration (in days):
"""
    extracted_preferences = llm_call(preferences_prompt)
    print("\nExtracted Preferences:")
    print(extracted_preferences)

    # Parse extracted preferences
    preferences = {}
    for line in extracted_preferences.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            preferences[key.strip()] = value.strip()

    # Validate extracted preferences
    required_keys = ["Destination", "Activities", "Budget", "Duration (in days)"]
    if not all(key in preferences for key in required_keys):
        print("\nCould not extract all required preferences. Please try again.")
        return

    # Fetch additional information
    weather = weather_tool(preferences["Destination"])
    print(f"\nWeather in {preferences['Destination']}: {weather}")

    origin = input("Please enter your departure city: ")
    flight_price = flight_price_estimator_tool(origin, preferences["Destination"])
    print(flight_price)

    # Plan itinerary
    itinerary_agent = ItineraryAgent()
    itinerary = itinerary_agent.plan_itinerary(
        extracted_preferences, int(preferences["Duration (in days)"])
    )
    print("\nPlanned Itinerary:")
    print(itinerary)

    # Currency conversion
    budget_amount = float(preferences["Budget"].replace("$", "").replace(",", ""))
    converted_budget = currency_converter_tool(budget_amount, "USD", "INR")
    if converted_budget:
        print(f"\nBudget in INR: {converted_budget:.2f} INR")
    else:
        print("\nCurrency conversion not available.")

    # Generate travel summary
    summary_prompt = f"""
Summarize the following travel plan:

Destination: {preferences['Destination']}
Activities: {preferences['Activities']}
Budget: {preferences['Budget']}
Duration: {preferences['Duration (in days)']} days
Itinerary: {itinerary}
Weather: {weather}
Flight Price: {flight_price}

Travel Summary:
"""
    travel_summary = llm_call(summary_prompt, max_tokens=2048)
    print("\nTravel Summary:")
    print(travel_summary)

In [37]:
# Entry point of the program
if __name__ == "__main__":
    main()# Call the main function to start the travel agent planner

    tracer.stop() # Stop the tracer when the analysis is complete

Welcome to the Personalized Travel Planner!



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Extracted Preferences:
Destination: India  
Activities: Explore mountains  
Budget: $1000  
Duration (in days): 10

Weather in India: Clear sky, 10.0°C
Estimated price from Dhaka to India: $500-$1000


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Planned Itinerary:
**10-Day Travel Itinerary: Exploring the Mountains of India**

**Destination:** Himachal Pradesh, India  
**Budget:** $1000  
**Duration:** 10 Days

---

### Day 1: Arrival in Delhi
- **Morning:** Arrive at Indira Gandhi International Airport, Delhi.
- **Afternoon:** Take a domestic flight or overnight bus to Shimla (approx. 7-hour journey).
- **Evening:** Check into a budget hotel or guesthouse in Shimla. 
- **Dinner:** Try local Himachali cuisine at a nearby restaurant.

### Day 2: Shimla Sightseeing
- **Morning:** Visit the Ridge, Mall Road, and Christ Church.
- **Afternoon:** Explore Jakhoo Temple for panoramic views of the Himalayas.
- **Evening:** Stroll along Mall Road for shopping and local snacks.
- **Overnight:** Stay in Shimla.

### Day 3: Shimla to Manali
- **Morning:** Depart early for Manali (approx. 7 hours by bus).
- **Afternoon:** Check into a budget hotel or homestay in Manali.
- **Evening:** Explore the local market and enjoy dinner at a local caf

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 15
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 14



Travel Summary:
**Travel Summary: Exploring the Mountains of India**

- **Destination:** Himachal Pradesh, India
- **Budget:** $1000
- **Duration:** 10 Days

### Itinerary Overview:

**Day 1:** Arrive in Delhi, then travel to Shimla. Check into budget accommodation and enjoy local cuisine for dinner.

**Day 2:** Sightseeing in Shimla, visiting key attractions like the Ridge, Mall Road, and Jakhoo Temple. Overnight in Shimla.

**Day 3:** Travel to Manali, check into accommodation, explore the local market, and dine at a café. Overnight in Manali.

**Day 4:** Adventure day in Solang Valley with activities like paragliding. Evening stroll along the Beas River. Overnight in Manali.

**Day 5:** Early trip to Rohtang Pass (if accessible) for snow activities and scenic views. Relax and overnight in Manali. 

**Day 6:** Depart for Dharamshala, check into accommodation, and visit the Dalai Lama Temple complex. Overnight in Dharamshala.

**Day 7:** Hike to Bhagsu. 

This plan emphasizes explori